In [4]:
cd ..

/Users/nguyenhuyhai/20221/lab_blockchain/orchai_validator


In [1]:
from pyspark.sql import SparkSession, DataFrame, Window
import pyspark.sql.functions as F

In [2]:
spark = SparkSession.builder.appName("Python Spark SQL basic example").master("local[*]").getOrCreate()

In [3]:
def preprocess(df):
    num_val_blocks_df = (
            df.groupBy("block_height")
            .agg({"operator_address": "count"})
            .withColumnRenamed("count(operator_address)", "validators_count_per_block")
        )

In [4]:
df = spark.createDataFrame(
    [
        ("a",1,2,0.1,1),
        ("a",2,5,0.1,2),
        ("a",3,1,0.1,3),
        ("a",4,4,0.1,4),
        ("a",5,5,0.1,5),
        ("a",6,6,0.1,6),
        ("a",7,8,0.1,1),
        ("a",8,3,0.1,1),
        ("a",9,1,0.1,2),
        ("a",10,2,0.1,3),
        ("b",1,4,0.2,2),
        ("b",2,4,0.2,1), 
        ("b",3,20,0.2,2), 
        ("b",4,12,0.2,4), 
        ("b",5,12,0.2,3), 
        ("b",6,4,0.2,0), 
        ("b",7,4,0.2,2), 
        ("b",8,10,0.2,3), 
        ("b",9,4,0.2,4), 
        ("b",10,4,0.2,5),
        ("c",1,4,0.3,2),
        ("c",2,4,0.3,1), 
        ("c",3,4,0.3,2), 
        ("c",4,4,0.3,3),
        ("c",5,7,0.3,7),
        ("c",6,4,0.3,3), 
        ("c",7,4,0.3,3), 
        ("c",8,4,0.3,4), 
        ("c",9,4,0.3,2), 
        ("c",10,4,0.3,1),
        ("d",1,4,0.4,4),
        ("d",2,4,0.4,2), 
        ("d",3,4,0.4,1), 
        ("d",4,4,0.4,3),
        ("d",5,7,0.4,7),
        ("d",6,4,0.4,4), 
        ("d",7,4,0.4,3), 
        ("d",8,4,0.4,2), 
        ("d",9,4,0.4,1), 
        ("d",10,4,0.4,0),
    
    ],

    ["operator_address", "block_height", "delegators_token", "commission_rate", "predict"]
    )

In [5]:
df1 = df.orderBy("operator_address", "block_height")

In [7]:
df1.printSchema()

root
 |-- operator_address: string (nullable = true)
 |-- block_height: long (nullable = true)
 |-- delegators_token: long (nullable = true)
 |-- commission_rate: double (nullable = true)
 |-- predict: long (nullable = true)



In [8]:
df1.show(20, truncate=False)

Py4JJavaError: An error occurred while calling o46.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 6 in stage 1.0 failed 1 times, most recent failure: Lost task 6.0 in stage 1.0 (TID 14) (DragonSea.mshome.net executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:189)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:164)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:65)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:365)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:329)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrReadChe

In [65]:
C_R_BASE = 0.1
def create_APR(df, C_R_BASE):
    df = df.withColumn("1-c_r", (1 - F.col("commission_rate"))).orderBy("operator_address", "block_height")
    df = df.withColumn("APR", (1/(1-C_R_BASE) * (1- F.col("commission_rate"))))
    return df

def create_delta(df: DataFrame, block_num):
    w = Window.partitionBy('operator_address').orderBy('block_height').rowsBetween(Window.unboundedPreceding, Window.unboundedFollowing)
    df = df.withColumn('subtract_de_token', F.when(
        (F.col("delegators_token")-F.first("delegators_token").over(w))>0 \
        ,F.col("delegators_token")-F.first("delegators_token").over(w))\
        .otherwise(0))
    
    df_delta = (
            df.groupBy("operator_address") \
                .agg({"subtract_de_token": "mean", \
                     "APR": "min"})
                .withColumnRenamed("avg(subtract_de_token)", "real_delta")\
                .withColumnRenamed("min(APR)", "APR")
    )
    df_delta = df_delta.withColumn("block_num", F.lit(block_num))
    

    return df_delta

def create_real_APR(real_df):
    df = real_df.withColumn("delta_APR_multiply", F.col("APR") * F.col("real_delta"))
    realAPR_df = df.groupBy("block_num") \
                .agg({"delta_APR_multiply": "sum", \
                       "real_delta": "sum"}) \
                .withColumnRenamed("sum(delta_APR_multiply)", "sum_delta_APR_multiply")\
                .withColumnRenamed("sum(real_delta)", "sum_real_delta")
    realAPR_df = realAPR_df.withColumn("real_APR", F.col("sum_delta_APR_multiply") / F.col("sum_real_delta"))
    return realAPR_df

def creat_unreal_APR(predict_df, realAPRdf, block_num):
    
       df_percen = predict_df.withColumn("block_num", F.lit(block_num))
       df_percen = df_percen.join(realAPRdf, on="block_num", how="left")
       df_percen = df_percen.withColumn("unreal_delta", F.col("predict") * F.col("sum_real_delta"))
       
       df_percen = df_percen.withColumn("unreal_delta_APR_multiply", F.col("APR") * F.col("unreal_delta"))

       unrealAPR_df = df_percen.groupBy("block_num") \
                     .agg({"unreal_delta_APR_multiply": "sum", \
                            "unreal_delta": "sum"}) \
                     .withColumnRenamed("sum(unreal_delta_APR_multiply)", "sum_unreal_delta_APR_multiply")\
                     .withColumnRenamed("sum(unreal_delta)", "sum_unreal_delta")
       unrealAPR_df = unrealAPR_df.withColumn("unreal_APR", F.col("sum_unreal_delta_APR_multiply") / F.col("sum_unreal_delta"))

       return unrealAPR_df

def join_df(real_df, predict_df):
    final_df = real_df.join(predict_df, on="block_num", how="right")
    final_df = final_df.drop("sum_delta_APR_multiply")
    final_df = final_df.drop("sum_real_delta")
    final_df = final_df.drop("sum_unreal_delta_APR_multiply")
    final_df = final_df.drop("sum_unreal_delta")
    return final_df

def back_test(df, C_R_BASE,start_block, end_block, timestamp_block):
    for i in range(start_block, end_block+1):
        if i == start_block:
            APR_df = create_APR(df, C_R_BASE)
            real_df = APR_df.filter(F.col("block_height").between(i, i+ timestamp_block))

            real_APR_df = create_delta(real_df, i)
            real_APR_df = create_real_APR(real_APR_df)

            predict_df = APR_df.filter(F.col("block_height") == i)
            unreal_APR_df = creat_unreal_APR(predict_df, real_APR_df, i)
            final_df = join_df(real_APR_df, unreal_APR_df)
        else:
            APR_df = create_APR(df, C_R_BASE)
            real_df = APR_df.filter(F.col("block_height").between(i, i+ timestamp_block))

            real_APR_df = create_delta(real_df, i)
            real_APR_df = create_real_APR(real_APR_df)

            predict_df = APR_df.filter(F.col("block_height") == i)
            unreal_APR_df = creat_unreal_APR(predict_df, real_APR_df, i)
            merge_df = join_df(real_APR_df, unreal_APR_df)
            final_df = final_df.union(merge_df)
    
    return final_df




In [61]:
df = create_APR(df, C_R_BASE)
real_df = df.filter(F.col("block_height").between(6, 6+ 1))

real_APR_df = create_delta(real_df, 6)
real_APR_df = create_real_APR(real_APR_df)

predict_df = df.filter(F.col("block_height") == 6)
unreal_APR_df = creat_unreal_APR(predict_df, real_APR_df, 6)
final_df6 = join_df(real_APR_df, unreal_APR_df)

In [52]:
final_df.show(200)

+---------+--------+------------------+
|block_num|real_APR|        unreal_APR|
+---------+--------+------------------+
|        1|     1.0|0.7777777777777778|
+---------+--------+------------------+



In [62]:
final_df6.show(200)

+---------+--------+------------------+
|block_num|real_APR|        unreal_APR|
+---------+--------+------------------+
|        6|     1.0|0.8461538461538461|
+---------+--------+------------------+



In [66]:
final_df = back_test(df, C_R_BASE, 1, 7,1)

In [67]:
final_df.show(200)

+---------+-----------------+------------------+
|block_num|         real_APR|        unreal_APR|
+---------+-----------------+------------------+
|        1|              1.0|0.7777777777777778|
|        2|0.888888888888889|0.8333333333333334|
|        3|              1.0|             0.875|
|        4|0.761904761904762|0.8492063492063491|
|        5|              1.0|0.8080808080808081|
|        6|              1.0|0.8461538461538461|
|        7|0.888888888888889|0.7901234567901235|
+---------+-----------------+------------------+



In [257]:
# def create_delta(df: DataFrame, label_win_size: int):
#     assert label_win_size % 150 == 0, "150 must be divisible by label_win_size"
#     size = int(label_win_size // 150) -1 # As we'll shift data after combining the data
    
#     ### window is used for shifting "size" blocks and calculate the mean
#     window = Window.partitionBy("operator_address").orderBy("block_height").rangeBetween(0, size)
#     w1 = Window.partitionBy("operator_address").orderBy("block_height").rangeBetween(0, 0)
    
#     ### lag_window is used for getting the null blocks - the last blocks that do not change
#     # lag_window = Window.partitionBy("operator_address").orderBy("block_height")
    
#     w=Window.partitionBy('operator_address').orderBy('block_height').rowsBetween(Window.unboundedPreceding, Window.unboundedFollowing)

#     df = df.withColumn(
#         "new_delta", F.sum(F.col("delegators_token")-F.first("delegators_token").over(w)).over(window)).orderBy("block_height")

#     # F.sum(F.when((F.col("delegators_token")-F.first("delegators_token").over(w))>0,F.col("delegators_token")-F.first("delegators_token").over(w)).otherwise(0)).over(window)).orderBy("block_height")

#     # df = df.withColumn("real_delta", (F.col("new_delta") - F.col("delegators_token")))
#     df = df.na.drop()
#     return df

In [258]:
# |-- block_height: integer (nullable = true)
#  |-- operator_address: string (nullable = true)
#  |-- jailed: integer (nullable = true)
#  |-- status: string (nullable = true)
#  |-- tokens: decimal(38,0) (nullable = true)
#  |-- commission_rate: double (nullable = true)
#  |-- self_bonded: decimal(38,6) (nullable = true)
#  |-- delegators_token: decimal(38,6) (nullable = true)
#  |-- validators_count_per_block: long (nullable = true)
#  |-- total_token_amt_per_block: decimal(38,0) (nullable = true)
#  |-- total_self_bonded_amt_per_block: decimal(38,6) (nullable = true)
#  |-- voting_power_score: decimal(38,6) (nullable = true)
#  |-- commission_score: double (nullable = true)
#  |-- self_bonded_score: double (nullable = true)
#  |-- vote_score: double (nullable = true)
#  |-- score: double (nullable = true)
#  |-- label: double (nullable = true)

In [16]:
list = ["oraivaloper18tf4uwrkcd4qk87jz3n0ruhsdzeg3fmdsdaw04",
"oraivaloper102rpf06zhyrfps4qzvehqjtr7rh52us5z2959t",
"oraivaloper10z9f6539v0ge78xlm4yh7tddrvw445s6d7s2xq",
"oraivaloper1zwxeq68hunufljymhy0f0x7r2vvh9mu2hh8nt8",
"oraivaloper1yujklf8xlcngsdylgqj352pj9x3yvrrwf3spc8",
"oraivaloper1mrv57zj3dpfyc9yd5xptnz2tqfez9fss4c9r85",
"oraivaloper17zr98cwzfqdwh69r8v5nrktsalmgs5sa83gpd9",
"oraivaloper1n4xkr4k4rpynp020e8xfqwm2e95g7rzx7w6kgl",
"oraivaloper1nedr7579x9dp6s3rrt7n5a5s5qczpnnstanuk5",
"oraivaloper16spaytw7nhdrv9lar0quwj6eex8cw9pvmtynxd",
"oraivaloper1asz5wl5c2xt8y5kyp9r04v54zh77pq90qar7e8",
"oraivaloper1djm07np8dzyg4et3d7dqtr3692l80nggvl0edh",
"oraivaloper1ucx0gm8kca2zvyr9d39z249j62y2t8r0rwtmr6",
"oraivaloper1ch3ewye24zm094ygmxu5e4z7d0xre3vhthctpn",
"oraivaloper1w96q7rrd6edc3fpmw6zayzdf5ajzucrnw3pt3s",
"oraivaloper1v26tdegnk79edw7xkk2xh8qn89vy6qej6yhsev",
"oraivaloper1kh9vejqxqqccavtv2nf683mx0z85mfpd7q566q",
"oraivaloper1d2ths86a84rxpp7s9j64qgrypntw2pn7lndfgs",
"oraivaloper1xsptthm2ylfw0salut97ldfan2jt032nye7s00",
"oraivaloper1qv5jn7tueeqw7xqdn5rem7s09n7zletreera88",
"oraivaloper16e6cpk6ycddk6208fpaya7tmmardhvr7h40yqy",
"oraivaloper10n3fs6fkl4fp9dcsdfl2vl3ay7pk7snntfg7da",
"oraivaloper130jsl66rgss6eq7qur02yfr6tzppdvxglz7n7g",
"oraivaloper1yc9nysml8dxy447hp3aytr0nssr9pd9a47l7gx",
"oraivaloper109vcny07r3waj9sld4ejasjyal0rudskeax7uc",
"oraivaloper1h89umsrsstyeuet8kllwvf2tp630n77aymck78",
"oraivaloper1mxqeldsxg60t2y6gngpdm5jf3k96dnju5el96f",
"oraivaloper1rqq57xt5r5pnuguffcrltnvkul7n0jdxxdgey0",
"oraivaloper1uhcwtfntsvk8gpwfxltesyl4e28aalmq9v9z0x",
"oraivaloper1xesqr8vjvy34jhu027zd70ypl0nnev5euy9nyl",
"oraivaloper1cp0jml5fxkdvmajcwvkue9d0sym6s0vqly88hg",
"oraivaloper1u2344d8jwtsx5as7u5jw7vel28puh34q7d3y64",
"oraivaloper1f6q9wjn8qp3ll8y8ztd8290vtec2yxyxxygyy2",
"oraivaloper1h9gg3xavqdau6uy3r36vn4juvzsg0lqvszgtvc",
"oraivaloper14nz2pqskfv9kcez8u0a9gnnsgwjerzqxpmne0y",
"oraivaloper1m2d5uhr65p9vvlw2w29kajud5q529a76v22wyu",
"oraivaloper13ckyvg0ah9vuujtd49yner2ky92lej6nwjvrjv",
"oraivaloper12ru3276mkzuuay6vhmg3t6z9hpvrsnplm2994n",
"oraivaloper14vcw5qk0tdvknpa38wz46js5g7vrvut8ku5kaa"]

In [6]:
df = spark.read.parquet("data/percent.parquet")

In [53]:
from math import log
import pandas as pd

In [54]:
df =pd.read_parquet("data/etl_parquet_1m")

In [55]:
pd.set_option("display.max_rows", None)

In [56]:
df = df[(df['block_height']<= 8920823) & (df['block_height']% 14400 == 8920823% 14400)].sort_values('block_height').head(100)

In [57]:
df.columns

Index(['block_height', 'operator_address', 'tokens', 'commission_rate',
       'self_bonded', 'delegators_token', 'vote_score', 'voting_power_score',
       'commission_score', 'self_bonded_score', 'score', 'label'],
      dtype='object')

In [5]:
import pandas as pd
from math import log

In [37]:
# import pandas as pd
# from math import log
# List1
block_height = [
    1, 1, 1, 1,
    2, 2, 2, 2,
    3, 3, 3, 3,
    4, 4, 4, 4
]
  
# List2
operator_address = [
    'a', 'b', 'c', 'd',
    'a', 'b', 'c', 'd',
    'a', 'b', 'c', 'd',
    'a', 'b', 'c', 'd']


tokens = [
    11,12,14,15,
    11,12,14,15,
    11,12,14,15,
    11,12,14,15
]

delegators_token = [
    25,12,24,52,
    44,42,66,74,
    64,32,23,54,
    52,12,43,44
]

self_bonded = [
    11,12,14,15,
    21,22,23,24,
    11,12,14,15,
    31,34,32,33
]

score = [
    1,2,3,4,
    5,2,2,1,
    2,4,4,1,
    6,1,1,2

]

  
# get the list of tuples from two lists.
# and merge them by using zip().
list_of_tuples = list(zip(block_height, operator_address, tokens, delegators_token, self_bonded, score))
  
# Assign data to tuples.
list_of_tuples
  
  
# Converting lists of tuples into
# pandas Dataframe.
df = pd.DataFrame(list_of_tuples,
                  columns=['block_height', 'operator_address', 'tokens', 'delegators_token','self_bonded', 'score'])
  
# Print data.
df

,block_height,operator_address,tokens,delegators_token,self_bonded,score
0,1,a,11,25,11,1
1,1,b,12,12,12,2
2,1,c,14,24,14,3
3,1,d,15,52,15,4
4,2,a,11,44,21,5
5,2,b,12,42,22,2
6,2,c,14,66,23,2
7,2,d,15,74,24,1
8,3,a,11,64,11,2
9,3,b,12,32,12,4


In [38]:
log(0.0000000000000000000000000000000001)

-78.28789316179756

In [39]:
def cal_delta(df: pd.DataFrame):
    df              = df.sort_values("block_height")
    num             = df.shape[0]
    ud_0            = df["delegators_token"].iloc[0]
    df["delegators_token"] = df["delegators_token"].apply(lambda x: max(x - ud_0, 0))
    delta           = df["delegators_token"].sum() / (num - 1) 
    block_height    = df["block_height"].iloc[0]
    df              = df[df["block_height"] == block_height]
    df["delta"]     = delta     
    return df

In [40]:
def dec_score(df:pd.DataFrame, col:str):
    
    # df = df[(df['block_height']<= end) &(df['block_height'] >= start) &(df['block_height']% hop_size == start % hop_size)].sort_values('block_height')
    def calculate(df: pd.DataFrame):
        df['count'] = df[col].count()
        df['pi'] = df[col]/df[col].sum()
        # df['ln(pi)'] = df['pi'].apply(lambda x: -9999999999 if (x ==0) else log(x))
        df['ln(pi)'] = df['pi'].apply(lambda x:log(x))

        df['pi_mul_ln(pi)'] = df['pi'] * df['ln(pi)']
        df['total'] = df['pi_mul_ln(pi)'].sum()
        df['decentralized_score'] = df['total'].apply(lambda x: abs(x))/ df['count'].apply(lambda x: log(x))

        return df

    df = df.groupby('block_height').apply(calculate)
    df = df.groupby('block_height').agg({"decentralized_score":"mean"}).reset_index()
    return df

def preprocess(df:pd.DataFrame, predicted_col: str, start: int, end_time_stamp: int):
    df = df[(df['block_height']<= end_time_stamp) &(df['block_height'] >= start)]
    
    df = df.groupby("operator_address").apply(cal_delta)
    df = df.drop(['operator_address', 'delegators_token'], axis = 1).reset_index()
    df["unreal_tokens"] = df['self_bonded'] + df[predicted_col]*(df['delta'].sum())/(df[predicted_col].sum())
    return df
def decentralize_backtest(df:pd.DataFrame, predicted_col:str, start:int, end: int, time_stamp: int, hop_size:  int):
    for i in range(start, end+1, hop_size):
        if i == start:    
            preprocecss_df = preprocess(df, predicted_col, i, i+ time_stamp)

            df_real_dec = dec_score(preprocecss_df,'tokens').rename(columns={'decentralized_score': 'real_decentralized_score'})

            df_unreal_dec = dec_score(preprocecss_df,'unreal_tokens').rename(columns={'decentralized_score': 'unreal_decentralized_score'})

            df_dec_final = df_real_dec.merge(df_unreal_dec, on = 'block_height', how = 'left')
        else:
            preprocecss_df = preprocess(df, predicted_col, i, i+ time_stamp)

            df_real_dec = dec_score(preprocecss_df,'tokens').rename(columns={'decentralized_score': 'real_decentralized_score'})

            df_unreal_dec = dec_score(preprocecss_df,'unreal_tokens').rename(columns={'decentralized_score': 'unreal_decentralized_score'})
            df_tem = df_real_dec.merge(df_unreal_dec, on = 'block_height', how = 'left')
            df_dec_final = pd.concat([df_dec_final, df_tem])
    return df_dec_final

In [41]:
df_dec_final = decentralize_backtest(df,'score',1,3,1,1)

In [26]:
preprocecss_df =preprocecss_df = preprocess(df, "score", 1, 2)

In [27]:
preprocecss_df

,operator_address,level_1,block_height,tokens,self_bonded,score,delta,unreal_tokens
0,a,0,1,11,11,1,19.0,25.125
1,b,1,1,12,12,0,30.0,12.000
2,c,2,1,14,14,3,42.0,56.375
3,d,3,1,15,15,4,22.0,71.500


In [43]:
df_dec_final

,block_height,real_decentralized_score,unreal_decentralized_score
0,1,0.994647,0.955767
0,2,0.994647,0.998021
0,3,0.994647,0.992982


In [42]:
# df_test= dec_score(preprocecss_df,'score').rename(columns={'decentralized_score': 'unreal_decentralized_score'})

In [32]:
df_test

,block_height,unreal_decentralized_score
0,1,0.994647


In [23]:
df_dec_final['compare'] = (df_dec_final['unreal_decentralized_score'] >= df_dec_final['real_decentralized_score'])

df_dec_final['accuracy'] = (df_dec_final['unreal_decentralized_score'] >= df_dec_final['real_decentralized_score']) / df_dec_final.shape[0]

df_summarized = df_dec_final.agg({"accuracy":"sum"})

df_final = df_dec_final.drop('accuracy', axis = 1)


In [24]:
df_final

,block_height,real_decentralized_score,unreal_decentralized_score,compare
0,1,0.994647,0.870311,False
0,2,0.994647,0.991965,False
0,3,0.994647,0.974562,False


In [11]:
df_summarized

accuracy    0.333333
dtype: float64

In [242]:
df_final

,block_height,real_decentralized_score,unreal_decentralized_score,compare
0,1,0.994647,0.955767,False
0,2,0.994647,0.998021,True
0,3,0.994647,0.992982,False


In [296]:
df =pd.read_parquet("data/etl_parquet_1m")

In [297]:
df.head(10)

,block_height,operator_address,tokens,commission_rate,self_bonded,delegators_token,vote_score,voting_power_score,commission_score,self_bonded_score,score,label
0,8920523,oraivaloper102rpf06zhyrfps4qzvehqjtr7rh52us5z2...,521047279,0.03,2950100.000000,518097179.000000,1.0,0.997026,7.000000e-01,0.000000,18.176208,18.176208
1,8920523,oraivaloper109vcny07r3waj9sld4ejasjyal0rudskea...,2293369056,0.53,0.000000,2293369056.000000,0.0,0.986912,0.000000e+00,0.000000,0.000000,0.000000
2,8920523,oraivaloper10z9f6539v0ge78xlm4yh7tddrvw445s6d7...,31304300,0.03,0.000000,31304300.000000,0.0,0.999821,7.000000e-01,0.000000,0.000000,0.000000
3,8920523,oraivaloper14nz2pqskfv9kcez8u0a9gnnsgwjerzqxpm...,64421274142,0.03,3001247188.242227,61420026953.757773,1.0,0.000000,7.000000e-01,0.129168,10.458336,10.158336
4,8920523,oraivaloper14vcw5qk0tdvknpa38wz46js5g7vrvut8ku...,28069598256,1.00,1009988.000000,28068588268.000000,1.0,0.839809,0.000000e+00,0.000000,12.718472,12.718472
5,8920523,oraivaloper16e6cpk6ycddk6208fpaya7tmmardhvr7h4...,3693310298,0.05,1600159994.354978,2093150303.645022,1.0,0.978923,5.000000e-01,0.000000,16.831384,16.831384
6,8920523,oraivaloper1n4xkr4k4rpynp020e8xfqwm2e95g7rzx7w...,1800198649,0.03,460240582.000000,1339958067.000000,1.0,0.989726,7.000000e-01,0.000000,18.117808,18.117808
7,8920523,oraivaloper1ucx0gm8kca2zvyr9d39z249j62y2t8r0rw...,3291091519,0.03,1069500.063596,3290022018.936404,1.0,0.981218,7.000000e-01,0.000000,18.049744,17.749744
8,8920523,oraivaloper1xesqr8vjvy34jhu027zd70ypl0nnev5euy...,145121811714,0.03,6378650000.000000,138743161714.000000,1.0,0.000000,7.000000e-01,0.549287,11.298574,11.298574
9,8920523,oraivaloper130jsl66rgss6eq7qur02yfr6tzppdvxglz...,60595160,0.10,0.886197,60595159.113803,0.0,0.999654,1.569325e-08,0.000000,0.000000,0.000000


In [298]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 532071 entries, 0 to 532070
Data columns (total 12 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   block_height        532071 non-null  int32  
 1   operator_address    532071 non-null  object 
 2   tokens              532071 non-null  object 
 3   commission_rate     532071 non-null  float64
 4   self_bonded         532071 non-null  object 
 5   delegators_token    532071 non-null  object 
 6   vote_score          532071 non-null  float64
 7   voting_power_score  532071 non-null  object 
 8   commission_score    532071 non-null  float64
 9   self_bonded_score   532071 non-null  float64
 10  score               532071 non-null  float64
 11  label               532071 non-null  float64
dtypes: float64(6), int32(1), object(5)
memory usage: 46.7+ MB


In [301]:
df.columns

Index(['block_height', 'operator_address', 'tokens', 'commission_rate',
       'self_bonded', 'delegators_token', 'vote_score', 'voting_power_score',
       'commission_score', 'self_bonded_score', 'score', 'label'],
      dtype='object')

In [302]:
df1 = df.drop('operator_address',axis = 1)
df[df1.columns] = df[df1.columns].apply(pd.to_numeric)


In [303]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 532071 entries, 0 to 532070
Data columns (total 12 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   block_height        532071 non-null  int32  
 1   operator_address    532071 non-null  object 
 2   tokens              532071 non-null  float64
 3   commission_rate     532071 non-null  float64
 4   self_bonded         532071 non-null  float64
 5   delegators_token    532071 non-null  float64
 6   vote_score          532071 non-null  float64
 7   voting_power_score  532071 non-null  float64
 8   commission_score    532071 non-null  float64
 9   self_bonded_score   532071 non-null  float64
 10  score               532071 non-null  float64
 11  label               532071 non-null  float64
dtypes: float64(10), int32(1), object(1)
memory usage: 46.7+ MB


In [279]:
def cal_delta(df: pd.DataFrame):
    df              = df.sort_values("block_height")
    num             = df.shape[0]
    ud_0            = df["delegators_token"].iloc[0]
    df["delegators_token"] = df["delegators_token"].apply(lambda x: max(x - ud_0, 0))
    delta           = df["delegators_token"].sum() / (num - 1) 
    block_height    = df["block_height"].iloc[0]
    df              = df[df["block_height"] == block_height]
    df["delta"]     = delta     
    df              = df.drop(columns=["operator_address", "delegators_token"])
    return df
    
    

In [292]:
def dec_score(df:pd.DataFrame, col:str):
    
    # df = df[(df['block_height']<= end) &(df['block_height'] >= start) &(df['block_height']% hop_size == start % hop_size)].sort_values('block_height')
    def calculate(df: pd.DataFrame):
        df['count'] = df[col].count()
        df['pi'] = df[col]/df[col].sum()
        df['ln(pi)'] = df['pi'].apply(lambda x: log(abs(x)))
        df['pi_mul_ln(pi)'] = df['pi'] * df['ln(pi)']
        df['total'] = df['pi_mul_ln(pi)'].sum()
        df['decentralized_score'] = df['total'].apply(lambda x: abs(x))/ df['count'].apply(lambda x: log(x))

        return df

    df = df.groupby('block_height').apply(calculate)
    df = df.groupby('block_height').agg({"decentralized_score":"mean"}).reset_index()
    return df

def preprocess(df:pd.DataFrame, predicted_col: str, start: int, end_time_stamp: int):
    df = df[(df['block_height']<= end_time_stamp) &(df['block_height'] >= start)]
    
    df = df.groupby("operator_address").apply(cal_delta)
    # df = df.drop(['operator_address', 'delegators_token'], axis = 1).reset_index()
    df["unreal_tokens"] = df['self_bonded'] + df[predicted_col]*(df['delta'].sum())/(df[predicted_col].sum())
    return df


def back_test_decentralized(df:pd.DataFrame, start:int, end: int, hop_size: int,win_size: int, predicted_col:str):
    for i in range(start, end+1, hop_size):
        if i == start:    
            print(i)

            preprocecss_df = preprocess(df, predicted_col, i, i+ win_size)

            df_real_dec = dec_score(preprocecss_df,'tokens').rename(columns={'decentralized_score': 'real_decentralized_score'})

            df_unreal_dec = dec_score(preprocecss_df,'unreal_tokens').rename(columns={'decentralized_score': 'unreal_decentralized_score'})

            df_dec_final = df_real_dec.merge(df_unreal_dec, on = 'block_height', how = 'left')
        else:
            print(i)
            preprocecss_df = preprocess(df, predicted_col, i, i+ win_size)

            df_real_dec = dec_score(preprocecss_df,'tokens').rename(columns={'decentralized_score': 'real_decentralized_score'})

            df_unreal_dec = dec_score(preprocecss_df,'unreal_tokens').rename(columns={'decentralized_score': 'unreal_decentralized_score'})
            df_tem = df_real_dec.merge(df_unreal_dec, on = 'block_height', how = 'left')
            df_dec_final = pd.concat([df_dec_final, df_tem])

            df_dec_final['compare'] = (df_dec_final['unreal_decentralized_score'] >= df_dec_final['real_decentralized_score'])

            df_dec_final['accuracy'] = (df_dec_final['unreal_decentralized_score'] >= df_dec_final['real_decentralized_score']) / df_dec_final.shape[0]

            summarze = df_dec_final.agg({"accuracy":"sum"})

            df_final = df_dec_final.drop('accuracy', axis = 1)
    return summarze, df_final



In [291]:
log(10**15)

34.538776394910684

In [294]:
preprocess_df = preprocess(df, "score", 7160273, 7160273+432000)

In [295]:
preprocess_df

,,block_height,tokens,commission_rate,self_bonded,vote_score,voting_power_score,commission_score,self_bonded_score,score,label,delta,unreal_tokens
operator_address,,,,,,,,,,,,,
oraivaloper102rpf06zhyrfps4qzvehqjtr7rh52us5z2959t,155371,7160273,1.186871e+08,0.03,2.000000e+06,1.0,0.999223,7.000000e-01,0.000000,18.193784,18.193818,7.847668e+06,1.219851e+09
oraivaloper109vcny07r3waj9sld4ejasjyal0rudskeax7uc,155372,7160273,1.491114e+10,0.03,5.003800e+08,1.0,0.902360,7.000000e-01,0.000000,17.418880,17.424239,1.035639e+09,1.666360e+09
oraivaloper10n3fs6fkl4fp9dcsdfl2vl3ay7pk7snntfg7da,155400,7160273,2.146100e+07,0.05,1.000000e+06,1.0,0.999859,5.000000e-01,0.000000,16.998872,14.598886,2.345597e+06,1.138866e+09
oraivaloper10z9f6539v0ge78xlm4yh7tddrvw445s6d7s2xq,155373,7160273,4.515930e+07,0.03,0.000000e+00,0.0,0.999704,7.000000e-01,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00
oraivaloper12ru3276mkzuuay6vhmg3t6z9hpvrsnplm2994n,155401,7160273,4.390020e+10,0.03,9.725087e+08,1.0,0.000000,7.000000e-01,0.000000,10.200000,10.200000,2.357227e+08,1.655274e+09
oraivaloper130jsl66rgss6eq7qur02yfr6tzppdvxglz7n7g,155381,7160273,2.145965e+09,0.10,1.500835e+09,1.0,0.985948,1.569325e-08,0.000000,13.887584,13.888354,3.189517e+06,2.430438e+09
oraivaloper13ckyvg0ah9vuujtd49yner2ky92lej6nwjvrjv,155402,7160273,4.647395e+10,0.03,4.328772e+08,1.0,0.000000,7.000000e-01,0.000000,10.200000,10.200000,2.968319e+08,1.115642e+09
oraivaloper14nz2pqskfv9kcez8u0a9gnnsgwjerzqxpmne0y,155374,7160273,5.029981e+10,0.03,1.062053e+09,1.0,0.000000,7.000000e-01,0.000000,10.200000,10.200000,1.506528e+09,1.744818e+09
oraivaloper14vcw5qk0tdvknpa38wz46js5g7vrvut8ku5kaa,155375,7160273,2.855843e+10,1.00,1.009988e+06,1.0,0.812996,0.000000e+00,0.000000,12.503968,12.514236,1.866771e+06,8.379973e+08


In [285]:
df_unreal_dec = df_unreal_dec = dec_score(preprocess_df,'unreal_tokens').rename(columns={'decentralized_score': 'unreal_decentralized_score'})


In [286]:
df_unreal_dec

,block_height,unreal_decentralized_score
0,7073723,0.910036


In [293]:
summarize, df_decentralized = back_test_decentralized(df, start=7059473, end=9583823, hop_size=14400, win_size=432000, predicted_col="score")  
    #summarize is Series, df_centralized is dataframe
print(summarize[0])

7059473
7073873
7088273
7102673
7117073
7131473
7145873
7160273


ValueError: math domain error

In [250]:
decimal.Decimal

NameError: name 'decimal' is not defined